<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=be0da06f433862f6a10871f11a692449e4e49479055dc6a5439d6583a08a4ab3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-04 10:40:10
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.41 C
-------------------
Today PnL: 67.04 K (0.48%)
Current PnL: -17.24 L (-11.59%)
CY Booked + Current PnL: -7.38 L (-4.96%)
-------------------
Total profit:  3.23 L
Total loss:  -20.46 L
-------------------
Total Booked + Current PnL: 19.60 L (16.11%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.0%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.96 L (55.36%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.66,70.0,H-MC,7.15,142776.0,27840.0,8752.0,0.50,24.22,6.13,31.83,81.0,3.18,1.07,47.99,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.42,63.0,H-LC,9.29,173030.0,16853.0,9240.0,1.67,10.79,5.34,16.71,22.0,1.82,1.30,30.11,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,107.90,71.0,M-SC,6.64,89591.0,-11186.0,11280.0,0.30,-11.10,12.59,0.09,245.0,-0.99,0.67,17.70,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,51.0,M-LC,4.50,215233.0,13754.0,13452.0,0.01,6.83,6.25,13.51,60.0,1.02,1.61,20.28,XY25,NTT,BANKS
43,ITC,452.00,-36.66,60.0,H-LC,2.95,204402.0,4264.0,16618.0,1.52,2.13,8.13,10.44,4.0,0.26,1.53,7.81,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,KPIGREEN,731.35,4.80,44.0,H-SC,11.07,124828.0,-469.0,59468.0,-0.21,-0.37,47.64,47.09,141.0,-0.01,0.93,56.23,MH,ATH,POWER
31,HAVELLS,2069.16,-0.65,60.0,H-MC,2.06,246340.0,-1391.0,76439.0,0.83,-0.56,31.03,30.30,92.0,-0.02,1.84,12.66,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,0.22,40.0,H-LC,9.86,229220.0,-12704.0,75964.0,-0.42,-5.25,33.14,26.15,8.0,-0.17,1.72,8.32,X40,ATH,IT
58,RELIANCE,1533.00,-14.79,41.0,H-LC,7.51,212831.0,2465.0,26327.0,-0.60,1.17,12.37,13.68,37.0,0.09,1.59,17.89,XY25,NTT,REFINERIES
45,JIOFIN,387.00,-0.99,42.0,H-LC,13.97,219030.0,1624.0,51866.0,-0.26,0.75,23.68,24.61,48.0,0.03,1.64,55.97,XY24,BTT,FINANCE
41,INFY,2275.00,-19.60,44.0,H-LC,11.08,313046.0,158.0,171518.0,-0.64,0.05,54.79,54.87,3.0,0.00,2.34,6.70,X40,BTT,IT
1,ABB,7934.00,-40.64,48.0,H-LC,13.24,248232.0,-13387.0,132606.0,-0.33,-5.12,53.42,45.57,7.0,-0.10,1.86,5.62,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.78,52.0,H-LC,2.14,233460.0,-12691.0,91890.0,-0.06,-5.16,39.36,32.18,16.0,-0.14,1.75,29.03,X200,ATH,IT
52,NESTLEIND,1377.00,-6.95,66.0,H-LC,2.75,286393.0,20967.0,35828.0,2.44,7.90,12.51,21.40,11.0,0.59,2.14,15.81,XY25,NTT,FMCG
43,ITC,452.00,-36.66,60.0,H-LC,2.95,204402.0,4264.0,16618.0,1.52,2.13,8.13,10.44,4.0,0.26,1.53,7.81,X40,NTT,FMCG
19,CIPLA,1795.00,-18.75,59.0,H-LC,5.41,216912.0,12412.0,29001.0,0.31,6.07,13.37,20.25,10.0,0.43,1.62,15.33,X40N,BTT,PHARMA
85,VBL,671.64,-10.86,52.0,H-LC,7.12,321233.0,5391.0,107260.0,-0.35,1.71,33.39,35.67,5.0,0.05,2.41,15.80,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3651.35,70.0,M-SC,10.57,116914.0,-30454.0,82272.0,13.94,-20.67,70.37,35.16,236.0,-0.37,0.88,19.42,XY24,NTT,AUTO
16,CAMPUS,393.00,-22.75,64.0,M-SC,3.21,160828.0,-6947.0,62787.0,4.92,-4.14,39.04,33.28,210.0,-0.11,1.20,24.52,XY24,NTT,FOOTWEAR
22,COLPAL,3767.14,5.55,67.0,H-MC,9.10,192387.0,-19354.0,101446.0,3.55,-9.14,52.73,38.77,84.0,-0.19,1.44,14.51,XY25,ATH,FMCG
36,ICICIGI,2260.25,-19.65,45.0,H-MC,4.32,138291.0,2158.0,28972.0,2.57,1.58,20.95,22.86,91.0,0.07,1.04,14.40,X40,ATH,INSURANCE
52,NESTLEIND,1377.00,-6.95,66.0,H-LC,2.75,286393.0,20967.0,35828.0,2.44,7.90,12.51,21.40,11.0,0.59,2.14,15.81,XY25,NTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,64.0,L-SC,35.29,90029.0,-23520.0,63569.0,-2.33,-20.71,70.61,35.27,268.0,-0.37,0.67,120.35,XR,NTT,HOTELS
61,SAIL,228.00,39.53,57.0,M-MC,7.60,220209.0,-4753.0,172402.0,-1.39,-2.11,78.29,74.53,192.0,-0.03,1.65,27.85,XY24,BTT,STEEL
34,HINDZINC,730.22,17.65,56.0,M-LC,2.70,191270.0,-13806.0,126372.0,-1.10,-6.73,66.07,54.89,52.0,-0.11,1.43,15.74,X5K,ATH,METALS
62,SAMMAANCAP,326.00,-171.59,66.0,M-SC,6.74,82386.0,-19824.0,113215.0,-0.94,-19.40,137.42,91.37,208.0,-0.18,0.62,33.74,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,253.64,43.0,L-MC,11.49,43406.0,-33800.0,59197.0,-0.86,-43.78,136.38,32.89,258.0,-0.57,0.33,19.54,XR,NTT,BANKS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,107.90,71.0,M-SC,6.64,89591.0,-11186.0,11280.0,0.30,-11.10,12.59,0.09,245.0,-0.99,0.67,17.70,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,84.36,46.0,H-SC,13.51,116667.0,-25812.0,45337.0,0.37,-18.12,38.86,13.70,163.0,-0.57,0.87,37.70,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.97,57.0,H-SC,3.63,220248.0,-49419.0,85743.0,0.06,-18.33,38.93,13.47,138.0,-0.58,1.65,11.40,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,117.08,53.0,M-SC,3.30,142191.0,-32368.0,32405.0,0.33,-18.54,22.79,0.02,221.0,-1.00,1.06,23.68,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.81,46.0,H-SC,2.91,114534.0,-29269.0,56019.0,0.87,-20.35,48.91,18.60,149.0,-0.52,0.86,26.39,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-26.44,50.0,H-LC,14.49,292613.0,-53343.0,177206.0,0.50,-15.42,60.56,35.80,1.0,-0.30,2.19,3.66,X40,BTT,IT
41,INFY,2275.00,-19.60,44.0,H-LC,11.08,313046.0,158.0,171518.0,-0.64,0.05,54.79,54.87,3.0,0.00,2.34,6.70,X40,BTT,IT
43,ITC,452.00,-36.66,60.0,H-LC,2.95,204402.0,4264.0,16618.0,1.52,2.13,8.13,10.44,4.0,0.26,1.53,7.81,X40,NTT,FMCG
85,VBL,671.64,-10.86,52.0,H-LC,7.12,321233.0,5391.0,107260.0,-0.35,1.71,33.39,35.67,5.0,0.05,2.41,15.80,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,48.0,H-LC,13.24,248232.0,-13387.0,132606.0,-0.33,-5.12,53.42,45.57,7.0,-0.10,1.86,5.62,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.76,78.0,H-MC,20.39,188230.0,37317.0,19990.0,0.54,24.73,10.62,37.97,93.0,1.87,1.41,56.29,AR,ATH,AUTO
14,BLUESTARCO,2326.38,15.45,69.0,H-MC,3.45,199200.0,34530.0,33446.0,1.60,20.97,16.79,41.28,89.0,1.03,1.49,30.63,X40N,ATH,AC
25,DIXON,18940.15,-28.66,70.0,H-MC,7.15,142776.0,27840.0,8752.0,0.50,24.22,6.13,31.83,81.0,3.18,1.07,47.99,X40N,ATH,IT
26,DMART,5201.00,-5.64,72.0,H-LC,14.95,235543.0,51062.0,19315.0,1.02,27.68,8.20,38.14,38.0,2.64,1.76,41.22,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,15.45,69.0,H-MC,3.45,199200.0,34530.0,33446.0,1.60,20.97,16.79,41.28,89.0,1.03,1.49,30.63,X40N,ATH,AC
33,HEROMOTOCO,5949.07,1.76,78.0,H-MC,20.39,188230.0,37317.0,19990.0,0.54,24.73,10.62,37.97,93.0,1.87,1.41,56.29,AR,ATH,AUTO
86,VOLTAS,1918.49,1.58,64.0,H-MC,4.52,213105.0,21363.0,74672.0,-0.67,11.14,35.04,50.08,99.0,0.29,1.60,18.70,XY25,ATH,AC
23,DABUR,735.00,-0.95,68.0,H-MC,4.33,215592.0,18564.0,71059.0,1.71,9.42,32.96,45.49,102.0,0.26,1.61,23.69,XY24,BTT,FMCG
17,CAMS,5211.76,-6.49,47.0,H-SC,5.53,106400.0,4396.0,39528.0,-0.09,4.31,37.15,43.06,122.0,0.11,0.80,22.16,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.76,78.0,H-MC,20.39,188230.0,37317.0,19990.0,0.54,24.73,10.62,37.97,93.0,1.87,1.41,56.29,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.25,110049.0,9545.0,21096.0,0.25,9.50,19.17,30.49,79.0,0.45,0.82,44.98,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,4.60,102502.0,11004.0,67744.0,1.62,12.03,66.09,86.07,223.0,0.16,0.77,48.85,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.47,47.0,H-SC,12.49,135532.0,12196.0,121572.0,1.16,9.89,89.70,108.45,119.0,0.10,1.01,34.45,AR,ATH,MISC
88,WIPRO,420.00,-12.43,47.0,M-LC,6.93,153686.0,2741.0,106720.0,-0.67,1.82,69.44,72.51,53.0,0.03,1.15,7.95,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2236.79,53.0,H-SC,5.74,62875.0,-18215.0,26885.0,-0.63,-22.46,42.76,10.69,156.0,-0.68,0.47,27.42,OX40N,NTT,MISC
21,COFFEEDAY,80.00,-39.64,64.0,L-SC,35.29,90029.0,-23520.0,63569.0,-2.33,-20.71,70.61,35.27,268.0,-0.37,0.67,120.35,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-2.99,48.0,H-SC,15.91,90257.0,-10706.0,105240.0,-0.46,-10.60,116.60,93.63,148.0,-0.10,0.68,34.44,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.34,55.0,H-SC,13.29,94080.0,-3900.0,25505.0,0.35,-3.98,27.11,22.05,152.0,-0.15,0.70,36.75,XR,ATH,FINANCE
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.25,110049.0,9545.0,21096.0,0.25,9.50,19.17,30.49,79.0,0.45,0.82,44.98,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,64.0,L-SC,35.29,90029.0,-23520.0,63569.0,-2.33,-20.71,70.61,35.27,268.0,-0.37,0.67,120.35,XR,NTT,HOTELS
33,HEROMOTOCO,5949.07,1.76,78.0,H-MC,20.39,188230.0,37317.0,19990.0,0.54,24.73,10.62,37.97,93.0,1.87,1.41,56.29,AR,ATH,AUTO
57,RELAXO,1176.00,-29.52,66.0,H-SC,5.34,65376.0,-50162.0,85152.0,2.27,-43.42,130.25,30.28,136.0,-0.59,0.49,28.53,X40N,NTT,FOOTWEAR
14,BLUESTARCO,2326.38,15.45,69.0,H-MC,3.45,199200.0,34530.0,33446.0,1.60,20.97,16.79,41.28,89.0,1.03,1.49,30.63,X40N,ATH,AC
26,DMART,5201.00,-5.64,72.0,H-LC,14.95,235543.0,51062.0,19315.0,1.02,27.68,8.20,38.14,38.0,2.64,1.76,41.22,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.90
1,25,43.84
2,50,73.97


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.84
LC    36.31
MC    25.84
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.75
X40      15.63
X40N     14.48
XY25     10.08
AR        9.49
XR        9.08
OX40N     7.44
MH        1.75
X200      1.75
X5K       1.43
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.05
H-MC    22.85
H-SC    22.61
M-SC    13.64
M-LC     7.18
M-MC     2.66
L-SC     1.59
L-LC     1.08
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,1.94,31.79
IT,13.43,-15.17,72.27
FINANCE,7.40,-21.96,66.93
BANKS,7.39,-15.74,64.88
PAINTS,6.04,-14.29,39.87
MISC,5.86,-17.38,84.67
ELECTRICAL,5.73,-7.61,66.92
AUTO,4.77,-6.20,52.40
AC,3.82,5.04,27.12


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2758609.0,22
AR,1211452.0,10
XR,1081201.0,13
X40,879938.0,10
X40N,500191.0,11
OX40N,464726.0,10
XY25,405279.0,7
SR,196010.0,2
X5K,126372.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2755977.0,27
M-SC,1424371.0,17
H-LC,1372334.0,16
H-MC,1153897.0,17
M-LC,419446.0,5
M-MC,321996.0,2
L-SC,248675.0,3
L-MC,59197.0,1
L-LC,40339.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,922858.0,6
M-SC,XY24,840438.0,7
H-SC,AR,662006.0,5
H-LC,X40,590558.0,5
H-SC,XR,521835.0,6
H-MC,XY24,448549.0,4
H-LC,AR,370859.0,2
M-MC,XY24,321996.0,2
H-MC,X40,211888.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
